# DATA SCIENCE INTERN - LGMVIP -OCT 2021 - PRAGATI GUPTA
## TASK - HANDWRITTEN EQUATION SOLVER

### Importing necessary libraries and modules

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sns
import random
from numpy import argmax
import cv2
from PIL import Image
import os
from os import listdir
from os.path import isfile, join
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import *
import pickle
from keras.models import Model
from keras.datasets import mnist
from keras.optimizers import *
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import model_from_json
from keras import backend as K

K.image_data_format() # substitute for: keras.backend.image_dim_ordering()

'channels_last'

In [6]:
def load_images(folder):
    train_data=[]
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename),cv2.IMREAD_GRAYSCALE)
        img=~img
        if img is not None:
            ret,thresh=cv2.threshold(img,127,255,cv2.THRESH_BINARY)
            ctrs,ret=cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
            cnt=sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
            w=int(28)
            h=int(28)
            maxi=0
            for c in cnt:
                x,y,w,h=cv2.boundingRect(c)
                maxi=max(w*h,maxi)
                if maxi==w*h:
                    x_max=x
                    y_max=y
                    w_max=w
                    h_max=h
            im_crop= thresh[y_max:y_max+h_max+10, x_max:x_max+w_max+10]
            im_resize = cv2.resize(im_crop,(28,28))
            im_resize=np.reshape(im_resize,(784,1))
            train_data.append(im_resize)
    return train_data

### appending numbers to each image to recognize them 

In [7]:
data_sym = []

data_sym = load_images("dataset math sym\\extracted_images\\0\\")
for i in range(0,len(data_sym)):
    data_sym[i] = np.append(data_sym[i], ['0'])

In [8]:
len(data_sym)

5465

In [9]:
sym1 = load_images("dataset math sym\\extracted_images\\1\\")
for i in range(0,len(sym1)):
    sym1[i] = np.append(sym1[i], ['1'])

data_sym = np.concatenate((data_sym, sym1))

In [10]:
sym2 = load_images("dataset math sym\\extracted_images\\2\\")
for i in range(0,len(sym2)):
    sym2[i] = np.append(sym2[i], ['2'])

data_sym = np.concatenate((data_sym, sym2))

In [11]:
sym3 = load_images("dataset math sym\\extracted_images\\3\\")
for i in range(0,len(sym3)):
    sym3[i] = np.append(sym3[i], ['3'])

data_sym = np.concatenate((data_sym, sym3))

In [12]:
sym4 = load_images("dataset math sym\\extracted_images\\4\\")
for i in range(0,len(sym4)):
    sym4[i] = np.append(sym4[i], ['4'])

data_sym = np.concatenate((data_sym, sym4))

In [13]:
sym5 = load_images("dataset math sym\\extracted_images\\5\\")
for i in range(0,len(sym5)):
    sym5[i] = np.append(sym5[i], ['5'])

data_sym = np.concatenate((data_sym, sym5))

In [14]:
sym6 = load_images("dataset math sym\\extracted_images\\6\\")
for i in range(0,len(sym6)):
    sym6[i] = np.append(sym6[i], ['6'])

data_sym = np.concatenate((data_sym, sym6))

In [15]:
sym7 = load_images("dataset math sym\\extracted_images\\7\\")
for i in range(0,len(sym7)):
    sym7[i] = np.append(sym7[i], ['7'])

data_sym = np.concatenate((data_sym, sym7))

In [16]:
sym8 = load_images("dataset math sym\\extracted_images\\8\\")
for i in range(0,len(sym8)):
    sym8[i] = np.append(sym8[i], ['8'])

data_sym = np.concatenate((data_sym, sym8))

In [17]:
sym9 = load_images("dataset math sym\\extracted_images\\9\\")
for i in range(0,len(sym9)):
    sym9[i] = np.append(sym9[i], ['9'])

data_sym = np.concatenate((data_sym, sym9))

### + is 10 , -  is 11, = is 12, division is 13, multiplication is 14. 

In [18]:
sym10 = load_images("dataset math sym\\extracted_images\\+\\")
for i in range(0,len(sym10)):
    sym10[i] = np.append(sym10[i], ['10'])

data_sym = np.concatenate((data_sym, sym10))

In [19]:
sym11 = load_images("dataset math sym\\extracted_images\\-\\")
for i in range(0,len(sym11)):
    sym11[i] = np.append(sym11[i], ['11'])

data_sym = np.concatenate((data_sym, sym11))

In [20]:
sym12 = load_images("dataset math sym\\extracted_images\\=\\")
for i in range(0,len(sym12)):
    sym12[i] = np.append(sym12[i], ['12'])

data_sym = np.concatenate((data_sym, sym12))

In [21]:
sym13 = load_images("dataset math sym\\extracted_images\\div\\")
for i in range(0,len(sym13)):
    sym13[i] = np.append(sym13[i], ['13'])

data_sym = np.concatenate((data_sym, sym13))

In [22]:
sym14 = load_images("dataset math sym\\extracted_images\\x\\")
for i in range(0,len(sym14)):
    sym14[i] = np.append(sym14[i], ['14'])

data_sym = np.concatenate((data_sym, sym14))

In [23]:
len(data_sym)       # total images we have

168471

In [24]:
df = pd.DataFrame(data_sym,index=None)
df.to_csv('new_dataset.csv',index=False)

### training

In [25]:
df_new = pd.read_csv('new_dataset.csv', index_col=False)
df_new.sample(10)

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
167626,97,118,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,14
142874,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,11
39156,0,0,0,77,154,178,178,178,178,178,...,0,0,0,0,0,0,0,0,0,2
158739,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
116932,0,0,0,0,0,0,0,0,0,79,...,0,0,0,0,0,0,0,0,0,10
141270,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11
31676,255,218,109,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
151946,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
85577,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
78423,0,0,0,0,0,0,232,0,0,0,...,0,0,0,0,0,0,0,0,0,5


In [26]:
df_new.shape

(168471, 785)

In [27]:
# col no 784 recognizes each img as which number the img represents
num_label = df_new[['784']]

num_label = np.array(num_label)

#  dropping col no 784 from original dataset
df_new.drop(df_new.columns[[784]],axis=1,inplace=True)

In [28]:
cat=to_categorical(num_label ,num_classes = 15)  # classes was 13
print(cat)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [29]:
cat.shape

(168471, 15)

In [38]:
l=[]
for i in range(50000):
    l.append(np.array(df_new[i:i+1]).reshape(1,28,28))
    
np.random.seed(7)
x_train = df_new.to_numpy().reshape(df_new.to_numpy().shape[0],28,28,1)

In [39]:
model = Sequential()
model.add(Conv2D(32, (3,3), input_shape=(28, 28,1), activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(15, activation='softmax'))   # was earlier 13
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, cat, epochs=5, batch_size=200,shuffle=True,verbose=1)

Epoch 1/5
843/843 [==============================] - 51s 60ms/step - loss: 0.5441 - accuracy: 0.8451
Epoch 2/5
843/843 [==============================] - 49s 59ms/step - loss: 0.1951 - accuracy: 0.9279
Epoch 3/5
843/843 [==============================] - 47s 55ms/step - loss: 0.1584 - accuracy: 0.9402
Epoch 4/5
843/843 [==============================] - 49s 59ms/step - loss: 0.1398 - accuracy: 0.9468
Epoch 5/5
843/843 [==============================] - 49s 58ms/step - loss: 0.1296 - accuracy: 0.9502


In [41]:
json_f=  model.to_json()
with open("task3_model.json", "w") as json_file:
    json_file.write(json_f)
model.save_weights("task3_model.h5")

### loading json file

In [56]:
json_file = open('task3_model.json', 'r')
file2 = json_file.read()
json_file.close()
file3 = model_from_json(file2)

file3.load_weights("task3_model.h5")

In [91]:
def check_img(img):
    if img is not None:
        img=~img
        ret,thresh=cv2.threshold(img,127,255,cv2.THRESH_BINARY)
        ctrs,ret=cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        cnt=sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
        w=int(28)
        h=int(28)
        train_data=[]
        rects=[]
        for c in cnt :
            x,y,w,h= cv2.boundingRect(c)
            rect=[x,y,w,h]
            rects.append(rect)
        bool_rect=[]
        for r in rects:
            l=[]
            for rec in rects:
                flag=0
                if rec!=r:
                    if r[0]<(rec[0]+rec[2]+10) and rec[0]<(r[0]+r[2]+10) and r[1]<(rec[1]+rec[3]+10) and rec[1]<(r[1]+r[3]+10):
                        flag=1
                    l.append(flag)
                if rec==r:
                    l.append(0)
            bool_rect.append(l)
        dump_rect=[]
        for i in range(0,len(cnt)):
            for j in range(0,len(cnt)):
                if bool_rect[i][j]==1:
                    area1=rects[i][2]*rects[i][3]
                    area2=rects[j][2]*rects[j][3]
                    if(area1==min(area1,area2)):
                        dump_rect.append(rects[i])
        final_rect=[i for i in rects if i not in dump_rect]
        for r in final_rect:
            x=r[0]
            y=r[1]
            w=r[2]
            h=r[3]
            im_crop =thresh[y:y+h+10,x:x+w+10]
            im_resize = cv2.resize(im_crop,(28,28))
            im_resize=np.reshape(im_resize,(28,28,1))
            train_data.append(im_resize)
    s=''
    for i in range(len(train_data)):
        train_data[i]=np.array(train_data[i])
        train_data[i]=train_data[i].reshape(1,28,28,1)
        result=np.argmax(file3.predict(train_data[i]), axis=-1)
        if(result[0]==10):
            s=s+'+'
        if(result[0]==11):
            s=s+'-'
        if(result[0]==12):
            s=s+'='
        if(result[0]==13):
            s=s+'/'
        if(result[0]==14):
            s=s+'*'
        if(result[0]==0):
            s=s+'0'
        if(result[0]==1):
            s=s+'1'
        if(result[0]==2):
            s=s+'2'
        if(result[0]==3):
            s=s+'3'
        if(result[0]==4):
            s=s+'4'
        if(result[0]==5):
            s=s+'5'
        if(result[0]==6):
            s=s+'6'
        if(result[0]==7):
            s=s+'7'
        if(result[0]==8):
            s=s+'8'
        if(result[0]==9):
            s=s+'9'
    
    #print(s)
    return s

In [92]:
img1 = cv2.imread('testimg1.png',cv2.IMREAD_GRAYSCALE)
s = check_img(img1)

In [93]:
print(s)

546+320


In [94]:
import re

res = sum(map(int, re.findall(r'[+-]?\d+', str(s))))

print(str(res))

866


In [95]:
img2= cv2.imread('testimg2.png',cv2.IMREAD_GRAYSCALE)
s2 = check_img(img2)
print(s2)
eval(s2)

88*22


1936